In [3]:
# importacoes
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import openpyxl
import time
import pandas as pd

In [6]:
# definindo lista de empresas:
lista_br = pd.read_excel('lista_hospitais_na_bolsa.xlsx')
tickers = lista_br['id_acao']
tickers

0    AALR3
1    DASA3
2    FLRY3
3    HAPV3
4    KRSA3
5    MATD3
6    ODPV3
7    ONCO3
8    QUAL3
9    RDOR3
Name: id_acao, dtype: object

In [7]:
# - 1 utilização do webdriver

browser = webdriver.Chrome()
time.sleep(2)
browser.get('https://statusinvest.com.br/acoes/')
time.sleep(15)


In [8]:
# remover propagandas
ad_close = browser.find_element(By.XPATH, '//button[@class="btn-close"]')
ad_close.click()
time.sleep(1)

In [9]:
# raspar os históricos de valuation
with pd.ExcelWriter('raspagem_valuation.xlsx', engine='openpyxl') as writer:
    for ticker in tickers:
        lupa = browser.find_element(By.XPATH, '//a[@class="main-search" and @title="Busca por nome de empresas, FIIs, índices e muito mais"]')
        lupa.click()
        input =  browser.find_element(By.XPATH, '//input[@class="Typeahead-input input tt-input"]')
        input.send_keys(ticker)
        time.sleep(3)
        input.send_keys(Keys.ARROW_DOWN)
        input.send_keys(Keys.ENTER)
        time.sleep(4)
        
        botao_historico = browser.find_element(By.XPATH, '//button[@title="Histórico do ativo" and @type="button"]')
        botao_historico.click()
        time.sleep(2)

        anos = []
        table_history = browser.find_element(By.XPATH, '//div[@class="table-history w-100"]')
        div_historicos = table_history.find_element(By.XPATH, '//div[@class="tr w-100 d-flex justify-start"]') 
        div_anos = div_historicos.find_element(By.XPATH, '//div[@class="tr w-100 d-flex justify-start"]')# find_element -> pega a primeira div só com os anos
        divs_com_anos_txt = div_anos.find_elements(By.TAG_NAME, 'div') # divs individuais com anos no texto e nada mais
        #len(divs_com_anos_txt) # DETECTOU TAMANHO DA ARRAY DOS ANOS (QUANTIDADE DE COLUNAS!!)
        
        for elem in divs_com_anos_txt: # monta array com os anos
            anos.append(elem.text)
        anos
        
        # agora para as demais divs, abaixo dos anos. 

        divs_history = table_history.find_elements(By.XPATH, '//div[@class="tr w-100 d-flex justify-start asset-last"]')
        dy = divs_history[0].text.split('\n')
        dy
        
        pl = divs_history[1].text.split('\n')
        pl

        pratio = divs_history[2].text.split('\n')
        pratio

        pvp = divs_history[3].text.split('\n')
        pvp

        evebtida = divs_history[4].text.split('\n')
        evebtida

        evebit = divs_history[5].text.split('\n')
        evebit


        pebtida = divs_history[6].text.split('\n')
        pebtida

        pebit = divs_history[7].text.split('\n')
        pebit

        vpa = divs_history[8].text.split('\n')
        vpa

        pativo = divs_history[9].text.split('\n')
        pativo

        lpa = divs_history[10].text.split('\n')
        lpa

        psr = divs_history[11].text.split('\n')
        psr

        pcapgiro = divs_history[12].text.split('\n')
        pcapgiro

        pativoliq = divs_history[13].text.split('\n')
        pativo

        #Indicadores de Endividamento

        divliqpl = divs_history[14].text.split('\n')

        divliqebtida = divs_history[15].text.split('\n')

        divliqebit = divs_history[16].text.split('\n')

        plativos = divs_history[17].text.split('\n')

        passativ = divs_history[18].text.split('\n')

        liqcorrente = divs_history[19].text.split('\n')

        # Indicadores de Eficiencia

        mbruta = divs_history[20].text.split('\n')

        mebtida = divs_history[21].text.split('\n')

        mebit = divs_history[22].text.split('\n')

        mliquida = divs_history[23].text.split('\n')

        # Indicadores de Rentabilidade

        roe = divs_history[24].text.split('\n')

        roa = divs_history[25].text.split('\n')

        roic = divs_history[26].text.split('\n')

        giroativos = divs_history[27].text.split('\n')

        # Indicadores de Crescimento

        cagrreceita = divs_history[28].text.split('\n')

        cagrlucro = divs_history[29].text.split('\n')



        # construindo dataframes e salvando
        raspagem = {
            'Ano': anos,
            'D.Y': dy,
            'P/L':pl,
            'PEG RATIO':pratio,
            'P/VP':pvp,
            'EV/EBITDA':evebtida,
            'EV/EBIT':evebit,
            'P/EBITDA':pebtida,
            'P/EBIT':pebit,
            'VPA':vpa,
            'P/ATIVO':pativo,
            'LPA':lpa,
            'P/SRf':psr,
            'P/CAP. GIRO':pcapgiro,
            'P/ATIVO CIRC. LIQ.':pativoliq,
            'DÍV. LÍQUIDA/PL':divliqpl,
            'DÍV. LÍQUIDA/EBITDA':divliqebtida,
            'DÍV. LÍQUIDA/EBIT':divliqebit,
            'PL/ATIVOS':plativos,
            'PASSIVOS/ATIVOS':passativ,
            'LIQ. CORRENTE':liqcorrente,
            'M. BRUTA':mbruta,
            'M. EBITDA':mebtida,
            'M. EBIT':mebit,
            'M. LÍQUIDA':mliquida,
            'ROE':roe,
            'ROA':roa,
            'ROIC':roic,
            'GIRO ATIVOS':giroativos,
            'CAGR RECEITAS 5 ANOS':cagrreceita,
            'CAGR LUCROS 5 ANOS':cagrlucro,
        }

        # Enconrando nome e cnpj
        company_description = browser.find_element(By.XPATH, '//div[@class="company-description w-100 w-md-70 ml-md-5"]')
        nome_cnpj = company_description.text.split('\n')
        nome_empresa = nome_cnpj[0]
        cnpj_empresa = nome_cnpj[1]
        nome_empresa = nome_empresa.replace('/', '.')
        
        # NOME DA EMPRESA PARA PÔR EM CADA LINHA
        company_description = browser.find_element(By.XPATH, '//div[@class="company-description w-100 w-md-70 ml-md-5"]')
        nome_cnpj = company_description.text.split('\n')
        nome_empresa = nome_cnpj[0]
        nome_empresa = nome_empresa.replace('/', '.')
        cnpj_empresa = nome_cnpj[1]
        
        # ADICIONANDO NOME E CNPJ AO DATAFRAME
        
        raspagem_df = pd.DataFrame(raspagem)

        numero_linhas = raspagem_df.shape[0]
        # numero_linhas

        empresa = []
        for n in range(numero_linhas):
            empresa.append(nome_empresa)
        # empresa
        raspagem_df.insert(0, 'empresa', empresa)

        cnpj = []
        for n in range(numero_linhas):
            cnpj.append(cnpj_empresa)
        # cnpj

        raspagem_df.insert(1, 'cnpj', cnpj)
        # raspagem_df

        # index_qtd = raspagem_df.shape[0]
        # for i in range(index_qtd): #quantas linhas tiverem o df para dar append na coluna e depois acrescentar a coluna com nomes e cnpj
        #     empresa.append(nome_empresa) #nome_empresa e cnpj_empreas definido por raspagem no começo
        #     cnpj.append(cnpj_empresa)
        # # empresa
        # # cnpj
        # # Acrescentando coluna empresa e cnpj
        # raspagem_df['empresa'] = empresa
        # raspagem_df['cnpj'] = cnpj
        # print(raspagem_df)
        
        # Usar ExcelWriter para escrever em um arquivo Excel
        # Adicionar cada DataFrame ao arquivo Excel
        raspagem_df.to_excel(writer, sheet_name=nome_empresa[:18], header=True, index=False)
            
            